In [1]:
import pandas as pd
import numpy as np
import os
import kaggle
import ast
from sklearn.cluster import KMeans
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import pickle
from scipy import spatial
import operator
from ast import literal_eval

In [2]:
#from kaggle.api.kaggle_api_extended import KaggleApi
#api = KaggleApi()
#api.authenticate()

#api.dataset_download_files('shuyangli94/food-com-recipes-and-user-interactions')

In [3]:
os.chdir(r"D:\recipe_recommender\food-com-recipes-and-user-interactions")
interactions_train = pd.read_csv("interactions_train.csv")
pp_recipes = pd.read_csv("PP_recipes.csv")
pp_users = pd.read_csv("PP_users.csv")
raw_interactions = pd.read_csv("RAW_interactions.csv")
raw_recipes = pd.read_csv("RAW_recipes.csv")

In [4]:
interactions_train.head()

,user_id,recipe_id,date,rating,u,i
0,2046,4684,2000-02-25,5.0,22095,44367
1,2046,517,2000-02-25,5.0,22095,87844
2,1773,7435,2000-03-13,5.0,24732,138181
3,1773,278,2000-03-13,4.0,24732,93054
4,2046,3431,2000-04-07,5.0,22095,101723


In [5]:
pp_recipes.head()

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"
1,146223,96900,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."
2,312329,120056,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696..."
3,74301,168258,"[40480, 10025, 31156, 40481]","[[1270, 1645, 28447], [21601], [27952, 29471, ...","[40480, 40482, 5539, 21601, 1073, 903, 2324, 4...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[7940, 3609, 7060, 6265, 1170, 6654, 5003, 3561]"
4,76272,109030,"[40480, 17841, 252, 782, 2373, 1641, 2373, 252...","[[1430, 11434], [1430, 17027], [1615, 23, 695,...","[40480, 40482, 14046, 1430, 11434, 488, 17027,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[3484, 6324, 7594, 243]"


In [6]:
id_dict = {}

for i in range(len(pp_recipes)):
    id_dict[pp_recipes['i'][i]] = pp_recipes['id'][i]


In [7]:
pp_recipes.head()

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"
1,146223,96900,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."
2,312329,120056,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696..."
3,74301,168258,"[40480, 10025, 31156, 40481]","[[1270, 1645, 28447], [21601], [27952, 29471, ...","[40480, 40482, 5539, 21601, 1073, 903, 2324, 4...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[7940, 3609, 7060, 6265, 1170, 6654, 5003, 3561]"
4,76272,109030,"[40480, 17841, 252, 782, 2373, 1641, 2373, 252...","[[1430, 11434], [1430, 17027], [1615, 23, 695,...","[40480, 40482, 14046, 1430, 11434, 488, 17027,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[3484, 6324, 7594, 243]"


In [8]:
raw_recipes[raw_recipes['id'] == 137739]['ingredients']

0    ['winter squash', 'mexican seasoning', 'mixed ...
Name: ingredients, dtype: object

Turn each item into a row making item (the recipe) the target variable. The predictor variables will be n_steps, n_ingredients, ingredient_tokens, nutrition, minutes. A user will then fll out a form specifying the predictor variables and based on the predictor variables a model will predict what recipe they may like. 

In [38]:
new_df = pd.DataFrame()
recipes, ratings = [], []

for i in range(len(pp_users)):
    item = ast.literal_eval(pp_users['items'][i])
    rating = ast.literal_eval(pp_users['ratings'][i])

    for j in range(len(item)):
        recipes.append(id_dict[item[j]])
        ratings.append(rating[j])

new_df['recipe'] = recipes
new_df['rating'] = ratings

In [39]:
new_df2 = new_df.groupby(['recipe']).mean()
new_df2 = new_df2.reset_index(drop=False)
new_df2 = new_df2[new_df2['rating'] > 3]

raw_recipes2 = raw_recipes[raw_recipes['id'].isin(list(new_df['recipe']))]
raw_recipes2 = raw_recipes2.reset_index(drop=True)

new_df2 = new_df2.reset_index(drop=True)

The predictor variables will be n_steps, n_ingredients, ingredient_tokens, nutrition, minutes

In [41]:
nutrition, minutes, ingreds, tags = {},{},{},{}

for i in range(len(raw_recipes2)):
    recipe = raw_recipes2['id'][i]
    key_words = ast.literal_eval(raw_recipes2['tags'][i])
    ingredients = ast.literal_eval(raw_recipes2['ingredients'][i])
    nutr = raw_recipes2['nutrition'][i]
    mins = raw_recipes2['minutes'][i]

    
    nutrition[recipe] = nutr
    minutes[recipe] = mins
    ingreds[recipe] = ingredients
    tags[recipe] = key_words

nutrition2, minutes2, ingreds2, tags2 = [],[],[],[]

for j in range(len(new_df2)):
    recipe = new_df2['recipe'][j]

    tags2.append(tags[recipe])
    nutrition2.append(nutrition[recipe])
    minutes2.append(minutes[recipe])
    ingreds2.append(ingreds[recipe])


new_df2['tags'] = tags2
new_df2['nutrition'] = nutrition2
new_df2['minutes'] = minutes2
new_df2['ingredients'] = ingreds2


In [45]:
new_df2.head()

,recipe,rating,tags,nutrition,minutes,ingredients
0,38,4.333333,"[weeknight, time-to-make, course, preparation,...","[170.9, 3.0, 120.0, 1.0, 6.0, 6.0, 12.0]",1485,"[blueberries, granulated sugar, vanilla yogurt..."
1,40,4.800000,"[60-minutes-or-less, time-to-make, course, pre...","[311.1, 0.0, 308.0, 0.0, 0.0, 0.0, 27.0]",35,"[sugar, lemons, rind of, fresh water, fresh le..."
2,45,4.000000,"[weeknight, time-to-make, course, main-ingredi...","[228.0, 10.0, 98.0, 11.0, 8.0, 8.0, 12.0]",80,"[sugar, margarine, egg, egg whites, flour, sal..."
3,46,5.000000,"[30-minutes-or-less, time-to-make, course, mai...","[4.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",25,"[rice vinegar, thangkwa, hom daeng, nam som pa..."
4,49,4.000000,"[weeknight, time-to-make, course, main-ingredi...","[627.7, 38.0, 8.0, 35.0, 115.0, 64.0, 4.0]",75,"[fresh mushrooms, butter, boneless skinless ch..."


In [46]:
# All recipes in df are rated > 3
new_df2 = new_df2.drop('rating',axis=1)

,recipe,tags,nutrition,minutes,ingredients
0,38,"[weeknight, time-to-make, course, preparation,...","[170.9, 3.0, 120.0, 1.0, 6.0, 6.0, 12.0]",1485,"[blueberries, granulated sugar, vanilla yogurt..."
1,40,"[60-minutes-or-less, time-to-make, course, pre...","[311.1, 0.0, 308.0, 0.0, 0.0, 0.0, 27.0]",35,"[sugar, lemons, rind of, fresh water, fresh le..."
2,45,"[weeknight, time-to-make, course, main-ingredi...","[228.0, 10.0, 98.0, 11.0, 8.0, 8.0, 12.0]",80,"[sugar, margarine, egg, egg whites, flour, sal..."
3,46,"[30-minutes-or-less, time-to-make, course, mai...","[4.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",25,"[rice vinegar, thangkwa, hom daeng, nam som pa..."
4,49,"[weeknight, time-to-make, course, main-ingredi...","[627.7, 38.0, 8.0, 35.0, 115.0, 64.0, 4.0]",75,"[fresh mushrooms, butter, boneless skinless ch..."


In [91]:
final_df = new_df2.copy()
final_df.head()

,recipe,tags,nutrition,minutes,ingredients
0,38,"[weeknight, time-to-make, course, preparation,...","[170.9, 3.0, 120.0, 1.0, 6.0, 6.0, 12.0]",1485,"[blueberries, granulated sugar, vanilla yogurt..."
1,40,"[60-minutes-or-less, time-to-make, course, pre...","[311.1, 0.0, 308.0, 0.0, 0.0, 0.0, 27.0]",35,"[sugar, lemons, rind of, fresh water, fresh le..."
2,45,"[weeknight, time-to-make, course, main-ingredi...","[228.0, 10.0, 98.0, 11.0, 8.0, 8.0, 12.0]",80,"[sugar, margarine, egg, egg whites, flour, sal..."
3,46,"[30-minutes-or-less, time-to-make, course, mai...","[4.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",25,"[rice vinegar, thangkwa, hom daeng, nam som pa..."
4,49,"[weeknight, time-to-make, course, main-ingredi...","[627.7, 38.0, 8.0, 35.0, 115.0, 64.0, 4.0]",75,"[fresh mushrooms, butter, boneless skinless ch..."


In [92]:
ings = []

for i in range(len(final_df)):
    #ing_list = ast.literal_eval(new_df2['ingredients'][i])
    ings.extend(final_df['ingredients'][i])

cnt = Counter(ings)
ingredient_list = sorted(cnt, key=cnt.get, reverse=True)[0:500]

#with open('ingredients.pickle', 'wb') as f:
#    pickle.dump(ingredient_list, f)

In [93]:
ethnicity = ['spanish','egyptian','tex-mex','welsh','australian','polynesian','pacific-northwest',
'swiss','italian','german','sudanese','chilean','costa-rican','hawaiian','malaysian','icelandic','norwegian',
'pennsylvania-dutch','somalian','pakistani','nepalese','venezuelan','asian','puerto-rican','nigerian',
'ethiopian','vietnamese','hungarian','laotian','dutch','chinese','south-african','brazilian','austrian',
'greek','middle-eastern','iraqi','ecuadorean','creole','african','korean','cambodian','caribbean','thai',
'indonesian','south-american','midwestern','north-american','mongolian','libyan','new-zealand','russian',
'congolese','mexican','south-west-pacific','japanese','native-american','portuguese','palestinian','cuban',
'scottish','central-american','filipino','indian','honduran','european','saudi-arabian','guatemalan','argentine',
'irish','turkish','georgian','british-columbian','peruvian','scandinavian','southern-united-states','lebanese',
'czech','canadian','cajun','northeastern-united-states','colombian','southwestern-united-states']

delete = ['cocktails','smoothies','beverages']

dietary_restrict = ['vegan','vegetarian','dairy-free','gluten-free','nut-free']

types_of_food = ['spreads','omelets-and-frittatas','frozen-desserts','dips','salsas',
'shakes','appetizers','sandwiches','cobblers-and-crisps','soul',
'seafood','pies','salads','barbecue','stews','side-dishes']

holiday = ['super-bowl','rosh-hashanah','new-years','mothers-day','st-patricks-day',
'fathers-day','valentines-day','ramadan','chinese-new-year','easter','christmas',
'april-fools-day','cinco-de-mayo','independence-day','labor-day','memorial-day',
'passover','kwanzaa','thanksgiving','hanukkah','halloween']


In [95]:
delete_recipes,ethnicity_lst,type_of_food_lst,holiday_lst,dr_lst = [],[],[],[],[]

for i in range(len(final_df)):
    tags = final_df['tags'][i]

    ethnicity_check = False
    type_of_food_check = False
    holiday_check = False
    dr_check = False

    ethnicity_lst2,type_of_food_lst2,holiday_lst2,dr_lst2 = [],[],[],[]

    for j in range(len(tags)):

        if tags[j] in delete:
            delete_recipes.append(final_df['recipe'][i])
            
        if tags[j] in ethnicity:
            ethnicity_lst2.append(tags[j])
            ethnicity_check = True

        if tags[j] in types_of_food:
            type_of_food_lst2.append(tags[j])
            type_of_food_check = True

        if tags[j] in holiday:
            holiday_lst2.append(tags[j])
            holiday_check = True

        if tags[j] in dietary_restrict:
            dr_lst2.append(tags[j])
            dr_lst_check = True


    if ethnicity_check == False:
        ethnicity_lst2.append("N/A")
    
    if type_of_food_check == False:
        type_of_food_lst2.append("N/A")

    if holiday_check == False:
        holiday_lst2.append("N/A")

    if dr_check == False:
        dr_lst2.append("N/A")

    ethnicity_lst.append(ethnicity_lst2)
    type_of_food_lst.append(type_of_food_lst2)
    holiday_lst.append(holiday_lst2)
    dr_lst.append(dr_lst2)


final_df['ethnicity'] = ethnicity_lst
final_df['type_of_food'] = type_of_food_lst
final_df['holiday'] = holiday_lst
final_df['dietary_restriction'] = dr_lst

final_df = final_df[~final_df['recipe'].isin(delete_recipes)]
final_df = final_df.reset_index(drop=True)

In [96]:
ingredients_bin = []

for i in range(len(final_df['ingredients'])):
    binaryList = []
    for ingred in ingredient_list:
        if ingred in final_df['ingredients'][i]:
            binaryList.append(1)
        else:
            binaryList.append(0)
            
    ingredients_bin.append(binaryList)
    
    

In [97]:
final_df['ingredients_bin'] = ingredients_bin
final_df = final_df.set_index("recipe")
final_df['nutrition'] = final_df.nutrition.apply(lambda x: literal_eval(x))
final_df = final_df[final_df['minutes'] < 3000]


In [98]:
final_df.head()

,tags,nutrition,minutes,ingredients,ethnicity,type_of_food,holiday,dietary_restriction,ingredients_bin
recipe,,,,,,,,,
38,"[weeknight, time-to-make, course, preparation,...","[170.9, 3.0, 120.0, 1.0, 6.0, 6.0, 12.0]",1485,"[blueberries, granulated sugar, vanilla yogurt...",[N/A],[frozen-desserts],[N/A],"[gluten-free, N/A]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
45,"[weeknight, time-to-make, course, main-ingredi...","[228.0, 10.0, 98.0, 11.0, 8.0, 8.0, 12.0]",80,"[sugar, margarine, egg, egg whites, flour, sal...",[north-american],[pies],[N/A],[N/A],"[1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, ..."
46,"[30-minutes-or-less, time-to-make, course, mai...","[4.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",25,"[rice vinegar, thangkwa, hom daeng, nam som pa...","[asian, thai]",[N/A],[N/A],"[gluten-free, N/A]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
49,"[weeknight, time-to-make, course, main-ingredi...","[627.7, 38.0, 8.0, 35.0, 115.0, 64.0, 4.0]",75,"[fresh mushrooms, butter, boneless skinless ch...","[european, italian]",[N/A],[N/A],[N/A],"[1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
58,"[weeknight, time-to-make, course, main-ingredi...","[280.1, 9.0, 36.0, 24.0, 59.0, 7.0, 7.0]",164,"[beef eye round, vegetable oil, dried thyme le...",[N/A],[stews],[N/A],"[gluten-free, N/A]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, ..."


In [99]:
final_df.shape


(144330, 9)

In [100]:
def Similarity(user_input, row1):

    ingreds_A = row1['ingredients_bin']
    ingreds_B = user_input['ingredients_bin'][user_input.index[0]]

    IngredDistance = spatial.distance.cosine(ingreds_A, ingreds_B)
    
    nutrition_A = row1['nutrition']
    nutrition_B = user_input['nutrition'][user_input.index[0]]

    NutDistance = spatial.distance.cosine(nutrition_A, nutrition_B)

    return IngredDistance + NutDistance 

In [101]:
def filter_df(df,time_filter, dietary_restriction, ethnicity, type_of_food, holiday):

    if time_filter == "Less than 5 Hours":
        df = df[df['minutes'] < 300]
    if time_filter == "Less than 3 Hours":
        df = df[df['minutes'] < 180]
    if time_filter == "Less than 1 Hour":
        df = df[df['minutes'] < 60]
    if time_filter == "Any Time":
        df = df

    if dietary_restriction != "None":
        df = df.explode('dietary_restriction')
        df = df[df['dietary_restriction'].isin(dietary_restriction)]

    if ethnicity != "Any":
        df = df.explode('ethnicity')
        df = df[df['ethnicity'] == ethnicity]

    if type_of_food != "Any":
        df = df.explode('type_of_food')
        df = df[df['type_of_food'] == type_of_food]

    if holiday != "No":
        df = df.explode('holiday')
        df = df[df['holiday'] == holiday]

    return df

In [102]:
def getNeighbors(user_input, df, K, name_df):
    distances = []
    
    for index, recipe in df.iterrows():
        
        dist = Similarity(user_input, recipe)
        name = name_df[name_df['id'] == index]['name']
        steps = name_df[name_df['id'] == index]['steps']
        distances.append((name,steps, dist))

    distances.sort(key=operator.itemgetter(2))
    neighbors = []

    for x in range(K):
        neighbors.append(distances[x])
    return neighbors

In [117]:
filtered_df = filter_df(final_df,"Less than 1 Hour","None","greek","Any","No")
filtered_df

,tags,nutrition,minutes,ingredients,ethnicity,type_of_food,holiday,dietary_restriction,ingredients_bin
recipe,,,,,,,,,
544,"[60-minutes-or-less, time-to-make, course, cui...","[36.1, 3.0, 6.0, 1.0, 3.0, 1.0, 1.0]",38,"[olive oil, mushroom, garlic cloves, basil, to...",greek,[salads],[N/A],"[gluten-free, N/A]","[1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
565,"[60-minutes-or-less, time-to-make, course, mai...","[501.8, 50.0, 80.0, 70.0, 49.0, 112.0, 8.0]",55,"[sweet red peppers, feta cheese, red wine vine...",greek,"[appetizers, dips]",[N/A],"[gluten-free, N/A]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
965,"[15-minutes-or-less, time-to-make, course, cui...","[504.3, 84.0, 0.0, 36.0, 5.0, 38.0, 0.0]",0,"[garlic, red wine vinegar, salt, black pepper,...",greek,[salads],[N/A],"[gluten-free, N/A]","[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2329,"[30-minutes-or-less, time-to-make, course, mai...","[236.8, 18.0, 19.0, 15.0, 13.0, 17.0, 8.0]",20,"[beef burgers, tabasco sauce, greek feta chees...",greek,[barbecue],[N/A],[N/A],"[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
3295,"[30-minutes-or-less, time-to-make, main-ingred...","[400.6, 18.0, 14.0, 33.0, 62.0, 21.0, 13.0]",20,"[turkey, pimento stuffed olives, dried parsley...",greek,[N/A],[N/A],[N/A],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...
504154,"[15-minutes-or-less, time-to-make, course, mai...","[291.7, 16.0, 8.0, 33.0, 20.0, 17.0, 13.0]",15,"[chickpeas, mayonnaise, feta cheese, fresh lem...",greek,[sandwiches],[N/A],"[vegetarian, N/A]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
505145,"[weeknight, 30-minutes-or-less, time-to-make, ...","[339.2, 29.0, 24.0, 22.0, 19.0, 24.0, 11.0]",20,"[roma tomatoes, red bell pepper, red onion, en...",greek,[salads],[N/A],"[vegetarian, N/A]","[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
513469,"[weeknight, 30-minutes-or-less, time-to-make, ...","[607.5, 40.0, 75.0, 76.0, 45.0, 39.0, 22.0]",30,"[extra virgin olive oil, italian sausage, onio...",greek,[N/A],[N/A],[N/A],"[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [118]:
raw_recipes2[raw_recipes2['id'] == 3295]['name']

67490    greek turkey burgers
Name: name, dtype: object

In [119]:
final_df2 = filtered_df[filtered_df.index != 3295]
user_input = filtered_df[filtered_df.index == 3295]

In [120]:
getNeighbors(user_input, final_df2 ,5, raw_recipes2)

d:\Anaconda\Lib\site-packages\scipy\spatial\distance.py:622: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / np.sqrt(uu * vv)


[(98559    my favorite sandwich   middle eastern style
  Name: name, dtype: object,
  98559    ['heat pita until warm', 'smear with hummus th...
  Name: steps, dtype: object,
  0.004135711176373902),
 (62223    fun  greek style  pasta salad
  Name: name, dtype: object,
  62223    ['boil and drail noodles', 'allow noodles to c...
  Name: steps, dtype: object,
  0.45642421251959187),
 (67303    greek potato salad
  Name: name, dtype: object,
  67303    ['boil potato slices 10-12 minutes or until fo...
  Name: steps, dtype: object,
  0.476622266138764),
 (6000    artichoke caviar
  Name: name, dtype: object,
  6000    ['combine all ingredients in a food processor'...
  Name: steps, dtype: object,
  0.5024214408449517),
 (67231    greek meatloaf with feta
  Name: name, dtype: object,
  67231    ['in a large bowl , stir the spices and crushe...
  Name: steps, dtype: object,
  0.5061397126754844)]